In [2]:
!pip install datasets

/bin/bash: /home/exx/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 92.7 MB/s eta 0:00:00:00:0100:01


In [16]:
pip install --upgrade Pillow


/bin/bash: /home/exx/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 50.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.2.0
    Uninstalling Pillow-9.2.0:
      Successfully uninstalled Pillow-9.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sam-2 1.0 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
sam-2 1.0 requires torch>=2.3.1, but you have torch 2.1.2 which is incompatible.
sam-2 1.0 requires torchvision>=0.18.1, but you have torchvision 0.16.2 which is incompatible.
gradio 4.16.0 requires pillow<11.0,>=8.0, but you have pillow 11.2.1 which is incompatible.
stable-baselines3 2.5.0 requires torch<3.0,>=2.3, but you have torch 2.1.2 which is incompatible.
Note: you may need to restart the kernel

In [42]:
!pip install timm


/bin/bash: /home/exx/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [ ]:
from datasets import load_dataset
import os

os.environ["HF_DATASETS_CACHE"] = "."

olives_train = load_dataset(
    "gOLIVES/OLIVES_Dataset", "biomarker_detection", split="train"
)
olives_test = load_dataset(
    "gOLIVES/OLIVES_Dataset", "biomarker_detection", split="test"
)

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
# For Train DATA
from torch.utils.data import DataLoader

# Covert into a Format Usable by Pytorch
olives = olives_train.with_format("torch")

# Example to get the VMT Biomarker of the first image in the dataset.
print(olives)

Dataset({
    features: ['Image', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'BCVA', 'CST', 'Eye_ID', 'Patient_ID'],
    num_rows: 78822
})


In [ ]:
# For Test DATA
olives_test = olives_test.with_format("torch")
print(olives_test)

Dataset({
    features: ['Image', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'BCVA', 'CST', 'Eye_ID', 'Patient_ID'],
    num_rows: 3871
})


In [ ]:
# did not separate by patient ID for now
from datasets import load_dataset

# Perform split directly on the HuggingFace Dataset object
split = olives.train_test_split(test_size=0.2, seed=42)

# Extract train and validation datasets
train_dataset = split["train"]
val_dataset = split["test"]

# Set PyTorch-compatible format
columns = ["Image", "BCVA", "CST", "B1", "B2", "B3", "B4", "B5", "B6"]
train_dataset.set_format("torch", columns=columns)
val_dataset.set_format("torch", columns=columns)

In [78]:
olives_test.set_format("torch", columns=columns)

In [ ]:
from torchvision import transforms

resize = transforms.Resize((224, 224))


def collate_fn(batch):
    filtered_batch = []
    for item in batch:
        biomarkers = torch.tensor(
            [
                float(item["B1"]) if item["B1"] == item["B1"] else float("nan"),
                float(item["B2"]) if item["B2"] == item["B2"] else float("nan"),
                float(item["B3"]) if item["B3"] == item["B3"] else float("nan"),
                float(item["B4"]) if item["B4"] == item["B4"] else float("nan"),
                float(item["B5"]) if item["B5"] == item["B5"] else float("nan"),
                float(item["B6"]) if item["B6"] == item["B6"] else float("nan"),
            ]
        )

        # Skip if all biomarkers are NaN
        if torch.isnan(biomarkers).all():
            continue
        item["biomarkers"] = torch.nan_to_num(biomarkers, nan=0.0)
        filtered_batch.append(item)

    if len(filtered_batch) == 0:
        return None, None, None

    images = torch.stack(
        [resize(item["Image"][0:1].float() / 255.0) for item in filtered_batch]
    )
    tabular = torch.stack(
        [
            torch.tensor(
                [
                    float(item["BCVA"]) if item["BCVA"] == item["BCVA"] else 0.0,
                    float(item["CST"]) if item["CST"] == item["CST"] else 0.0,
                ]
            )
            for item in filtered_batch
        ]
    )
    labels = torch.stack([item["biomarkers"] for item in filtered_batch])

    return images, tabular, labels


# Load Data
train_loader = DataLoader(
    train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn
)
val_loader = DataLoader(
    val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn
)

In [ ]:
# For testing data
test_loader = DataLoader(
    olives_test, batch_size=16, shuffle=False, collate_fn=collate_fn
)

In [81]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
# Try efficienct NET
import timm
import torch.nn as nn


class BiomarkerNet(nn.Module):
    def __init__(self):
        super(BiomarkerNet, self).__init__()

        # Load EfficientNetV2-M backbone with 1 channel input
        self.cnn = timm.create_model(
            "efficientnetv2_m",
            pretrained=True,
            in_chans=1,
            num_classes=0,  # we'll define our own head
        )  # Output feature dim: 1280

        # Tabular input for BCVA and CST
        self.tabular = nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), nn.Linear(16, 16), nn.ReLU()
        )

        # Combine CNN + tabular features
        self.classifier = nn.Sequential(
            nn.Linear(1280 + 16, 64), nn.ReLU(), nn.Linear(64, 6)  # 6 biomarkers
        )

    def forward(self, x_img, x_tab):
        x_img = self.cnn(x_img)  # Output shape: [B, 1280]
        x_tab = self.tabular(x_tab)  # Output shape: [B, 16]
        x = torch.cat((x_img, x_tab), dim=1)
        return self.classifier(x)

-   The code uses a modified ResNet-50 architecture to accept grayscale images and combines image features with tabular clinical data for biomarker prediction.
-   A similar model based on ResNet-18 was also tested, but only ResNet-50's implementation is shown here due to minimal differences between the two.
-   The main variation lies in the feature output size (512 for ResNet-18 vs. 2048 for ResNet-50), which affects the input dimension of the final classifier.


In [ ]:
# Try Resnet 50
from torchvision import models


class BiomarkerNet(nn.Module):
    def __init__(self):
        super(BiomarkerNet, self).__init__()

        # Load ResNet50 (note: pretrained uses RGB input by default)
        resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

        # Replace the first conv layer to accept grayscale (1 channel)
        resnet.conv1 = nn.Conv2d(
            1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        )

        # Remove the classifier head (fc layer)
        self.cnn = nn.Sequential(
            *list(resnet.children())[:-1]
        )  # Output: [B, 2048, 1, 1]

        # Flattened image features (ResNet50 outputs 2048-dim feature vector)
        self.img_fc = nn.Flatten()

        # Process tabular features (BCVA + CST)
        self.tabular = nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), nn.Linear(16, 16), nn.ReLU()
        )

        # Final classifier combining both
        self.classifier = nn.Sequential(
            nn.Linear(2048 + 16, 64), nn.ReLU(), nn.Linear(64, 6)  # 6 biomarker outputs
        )

    def forward(self, x_img, x_tab):
        x_img = self.cnn(x_img)  # [B, 2048, 1, 1]
        x_img = self.img_fc(x_img)  # [B, 2048]
        x_tab = self.tabular(x_tab)  # [B, 16]
        x = torch.cat((x_img, x_tab), dim=1)  # [B, 2064]
        return self.classifier(x)

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

model = BiomarkerNet().to(device)
criterion = nn.BCEWithLogitsLoss()  # no sigmoid in model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=100, eta_min=1e-6
)  # cosine annealing

No pretrained weights exist or were found for this model. Using random initialization.


In [ ]:
# TRAINING
for epoch in range(100):
    model.train()
    train_loss = 0.0
    for images, tabs, labels in train_loader:
        if images is None:
            continue
        images, tabs, labels = images.to(device), tabs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images, tabs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, tabs, labels in val_loader:
            if images is None:
                continue
            images, tabs, labels = images.to(device), tabs.to(device), labels.to(device)
            outputs = model(images, tabs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            y_true.append(labels.cpu().numpy())
            y_pred.append(torch.sigmoid(outputs).cpu().numpy())

    # Compute F1
    y_true = np.vstack(y_true)
    y_pred = np.vstack(y_pred)
    y_pred_binary = (y_pred >= 0.5).astype(int)
    f1 = f1_score(y_true, y_pred_binary, average="macro")

    scheduler.step()  # update LR
    # current_lr = scheduler.get_last_lr()[0]

    print(
        f"Epoch {epoch+1:03d} | Train Loss: {train_loss/len(train_loader):.4f} | "
        f"Val Loss: {val_loss/len(val_loader):.4f} | F1: {f1:.4f}"
    )

    # Save every 5 epochs
    if (epoch + 1) % 5 == 0:
        model_path = f"biomarker_resnet50_epoch_{epoch+1:03d}.pth"
        torch.save(model.state_dict(), model_path)
        print(f"Saved model to {model_path}")

Epoch 001 | Train Loss: 0.5420 | Val Loss: 0.4910 | F1: 0.5239


/home/exx/anaconda3/envs/oct_env/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 002 | Train Loss: 0.4807 | Val Loss: 0.4667 | F1: 0.5541


/home/exx/anaconda3/envs/oct_env/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 003 | Train Loss: 0.4282 | Val Loss: 0.3729 | F1: 0.7158


...

Epoch 098 | Train Loss: 0.0000 | Val Loss: 0.2586 | F1: 0.9702


/home/exx/anaconda3/envs/oct_env/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 099 | Train Loss: 0.0000 | Val Loss: 0.2505 | F1: 0.9702


/home/exx/anaconda3/envs/oct_env/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 100 | Train Loss: 0.0000 | Val Loss: 0.2553 | F1: 0.9708
Saved model to biomarker_resnet50_epoch_100.pth


In [ ]:
torch.save(model.state_dict(), "biomarker_model.pth")

In [ ]:
# Load model for testing
model = BiomarkerNet().to(device)
model.load_state_dict(torch.load("biomarker_epoch_100.pth"))
model.eval()

No pretrained weights exist or were found for this model. Using random initialization.


BiomarkerNet(
  (cnn): EfficientNet(
    (conv_stem): Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (drop_path): Identity()
        )
        (1): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, tabs, labels in test_loader:
        if images is None:
            continue
        images, tabs = images.to(device), tabs.to(device)
        outputs = model(images, tabs)
        probs = torch.sigmoid(outputs).cpu().numpy()
        y_pred.append(probs)
        y_true.append(labels.numpy())

# Stack everything
y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

# Convert probabilities to binary predictions
y_pred_binary = (y_pred >= 0.5).astype(int)

# Macro F1 Score
f1 = f1_score(y_true, y_pred_binary, average="macro")
print(f"Macro F1 Score on Test Set: {f1:.4f}")

df_pred = pd.DataFrame(y_pred_binary, columns=["B1", "B2", "B3", "B4", "B5", "B6"])
df_pred.to_csv("oct_test_predictions_binary.csv", index=False)

/home/exx/anaconda3/envs/oct_env/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Macro F1 Score on Test Set: 0.4994
